In [1]:
from src.data_loader.data_loader import BooksDataset
from src.models.mm_model import MmModel
from src.train import Trainer
import torch
import random
import numpy as np

C:\Users\driou\Desktop\LLMrec_ppd\notebooks


FileNotFoundError: [Errno 2] No such file or directory: './data/books/train.json'

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(2022)

In [3]:
dataset = BooksDataset(data_dir="../data/netflix")

In [ ]:
model = MmModel(n_users=dataset.n_users,n_items=dataset.n_items,adjacency_matrix=dataset.get_dataset("adjacency_matrix"),interactions= dataset.get_dataset("interactions") ,interactions_t= dataset.get_dataset("interactions_T") ,image_embeddings_data=dataset.get_dataset("images"),text_embeddings_data=dataset.get_dataset("text"),embed_size=64,n_layers=3,user_profiles_data=dataset.get_dataset("user_profiles"),book_attributes_data=dataset.get_dataset("books_attributes"))
model.to(device)

In [ ]:
trainer = Trainer(model=model,dataset=dataset,lr=0.0001)

In [ ]:
trainer.train(epochs=10,batch_size=1024)